In [1]:
from mosaicml import *
from mosaicml.constants import MLModelFlavours

Using TensorFlow backend.
/home/pcadmin/Desktop/project/AILogistics/mosaic-ai-client/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pcadmin/Desktop/project/AILogistics/mosaic-ai-client/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pcadmin/Desktop/project/AILogistics/mosaic-ai-client/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) /

In [2]:
import numpy as np
import os
import tensorflow as tf
from matplotlib import pyplot as plt
from PIL import Image
from tensordata import ops as utils_ops
from tensordata import config
# import gbiml
import inspect
import cloudpickle as cp

from tensordata import label_map_util
from tensordata import visualization_utils as vis_util
from tensordata import detection_utils as detection_utils
from werkzeug.datastructures import FileStorage

In [3]:
path = r'tensordata/aptifyfull_3_.jpg'
PATH_TO_FROZEN_GRAPH=r'tensordata/frozen_inference_graph.pb'
PATH_TO_LABELS=r'tensordata/label_map_table_detection.pbtxt'
NUM_CLASSES=config.num_labels
scoring_path = r'tensordata/score.pkl'
graph_path = r'tensordata/frozen_inference_graph.pb'

In [4]:
def load_frozen_inference_graph(path):
    detection_graph = tf.Graph()
    with detection_graph.as_default():
      od_graph_def = tf.GraphDef()
      with tf.gfile.GFile(path, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
    return detection_graph

def save_tf_model(graph,path):
    with tf.gfile.GFile(path, "wb") as f:
        f.write(graph.as_graph_def().SerializeToString())
        print("Frozen Graph Saved to :",path)
            

def save_scoring_func(scoring_func, path):
    with open(path, 'wb') as out:
        cp.dump(scoring_func, out)


def load_scoring_func(path):
    with open(path, 'rb') as f:
        scoring_func = cp.load(f)
    return scoring_func

def load_image_into_numpy_array(image):
    '''
    Returns a numpy array of the image

    Args:
    image     : PIL Image object
    '''
    if len(image.getbands())<3: image=image.convert('RGB')
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [5]:
detection_graph = load_frozen_inference_graph(graph_path)
#print(detection_graph)
#sf = load_scoring_func(scoring_path)

In [6]:
image = Image.open(path)
image_np = load_image_into_numpy_array(image)
image_np_expanded = np.expand_dims(image_np, axis=0)
#sf(detection_graph,image_np_expanded,threshold=0.0)
#image_np_expanded

In [7]:
image_np.shape

(2200, 1700, 3)

In [8]:
type(image_np)

numpy.ndarray

In [9]:
image_lst = image_np.tolist()

In [22]:
img_np_arry = np.array(image_lst)

In [23]:
img_np_arry.shape

(2200, 1700, 3)

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [8]:

def reframe_box_masks_to_image_masks(box_masks, boxes, image_height,
                                     image_width):
  """Transforms the box masks back to full image masks.

  Embeds masks in bounding boxes of larger masks whose shapes correspond to
  image shape.

  Args:
    box_masks: A tf.float32 tensor of size [num_masks, mask_height, mask_width].
    boxes: A tf.float32 tensor of size [num_masks, 4] containing the box
           corners. Row i contains [ymin, xmin, ymax, xmax] of the box
           corresponding to mask i. Note that the box corners are in
           normalized coordinates.
    image_height: Image height. The output mask will have the same height as
                  the image height.
    image_width: Image width. The output mask will have the same width as the
                 image width.

  Returns:
    A tf.float32 tensor of size [num_masks, image_height, image_width].
  """
  # TODO(rathodv): Make this a public function.
  def reframe_box_masks_to_image_masks_default():
    """The default function when there are more than 0 box masks."""
    def transform_boxes_relative_to_boxes(boxes, reference_boxes):
      boxes = tf.reshape(boxes, [-1, 2, 2])
      min_corner = tf.expand_dims(reference_boxes[:, 0:2], 1)
      max_corner = tf.expand_dims(reference_boxes[:, 2:4], 1)
      transformed_boxes = (boxes - min_corner) / (max_corner - min_corner)
      return tf.reshape(transformed_boxes, [-1, 4])

    box_masks_expanded = tf.expand_dims(box_masks, axis=3)
    num_boxes = tf.shape(box_masks_expanded)[0]
    unit_boxes = tf.concat(
        [tf.zeros([num_boxes, 2]), tf.ones([num_boxes, 2])], axis=1)
    reverse_boxes = transform_boxes_relative_to_boxes(unit_boxes, boxes)
    return tf.image.crop_and_resize(
        image=box_masks_expanded,
        boxes=reverse_boxes,
        box_ind=tf.range(num_boxes),
        crop_size=[image_height, image_width],
        extrapolation_value=0.0)
  image_masks = tf.cond(
      tf.shape(box_masks)[0] > 0,
      reframe_box_masks_to_image_masks_default,
      lambda: tf.zeros([0, image_height, image_width, 1], dtype=tf.float32))
  return tf.squeeze(image_masks, axis=3)

In [9]:
def run_inference_for_single_image(image, graph):
    '''
    Returns predictions with scores, bounding boxes, number of detections etc in a form of dictionary

    Args:
    image              : numpy array of size (height,width,3)
    graph              : frozen inference graph
    '''
    with graph.as_default():
      with tf.Session() as sess:
        # Get handles to input and output tensors
        ops = tf.get_default_graph().get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in [
            'num_detections', 'detection_boxes', 'detection_scores',
            'detection_classes', 'detection_masks'
        ]:
          tensor_name = key + ':0'
          if tensor_name in all_tensor_names:
            tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                tensor_name)
        if 'detection_masks' in tensor_dict:
          # The following processing is only for single image
          detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
          detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
          # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
          real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
          detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
          detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
          detection_masks_reframed =reframe_box_masks_to_image_masks(
              detection_masks, detection_boxes, image.shape[0], image.shape[1])
          detection_masks_reframed = tf.cast(
              tf.greater(detection_masks_reframed, 0.5), tf.uint8)
          # Follow the convention by adding back the batch dimension
          tensor_dict['detection_masks'] = tf.expand_dims(
              detection_masks_reframed, 0)
        image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

        # Run inference
        output_dict = sess.run(tensor_dict,
                               feed_dict={image_tensor: np.expand_dims(image, 0)})

        # all outputs are float32 numpy arrays, so convert types as appropriate
        output_dict['num_detections'] = int(output_dict['num_detections'][0])
        output_dict['detection_classes'] = output_dict[
            'detection_classes'][0].astype(np.uint8)
        output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
        output_dict['detection_scores'] = output_dict['detection_scores'][0]
        if 'detection_masks' in output_dict:
          output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [10]:
@scoring_func
def score(detection_graph, request):
    '''
    It finds out the regions of interest from the images specified by image_loc parameter 
    Args:
    image_loc          : either a string (single path) of a list of paths to input images
    category_index     : category index dictionary to map class ids with class names
    threshold          : minimum confidence level in order to consider prediction.
                         Default=0.97
    '''
    global category_index
    out_dir = None
    threshold = .97
    #image
    file= FileStorage(request.files['file1'])
    file.save("f2.jpg")
    image_req = Image.open("f2.jpg")
    image_req_np = load_image_into_numpy_array(image_req)
    img = image_req_np
    
    # list
    #import pdb; pdb.set_trace()
    #file= FileStorage(request.files['file1'])
    #file.save("f2.txt")
    #with open("f2.txt","r") as f:
    #    image_req=f.read()
    #img = np.asarray(image_req)
    
    # for payload
    #req_data = request.json["payload"]
    #data_list = req_data
    #img = np.asarray(data_list)
    
    # Process the images
    output_dict = run_inference_for_single_image(img, detection_graph)

    # Setting custom threshold for the predictions and extracting only the top prediction per class
    output_dict['detection_scores']=np.where(output_dict['detection_scores']>threshold,
                                             output_dict['detection_scores'],
                                             0.0)
    final_output = {}
    boxes = output_dict['detection_boxes']
    scores = output_dict['detection_scores']
    classes = output_dict['detection_classes']

    for box,score,cls in zip(boxes,scores,classes):
        if score<0.5:
            continue
        cls = category_index[cls]['name']
        if cls not in  final_output.keys():
            final_output[cls]=[[box.tolist(),score]]
        else:
            final_output[cls].append([box.tolist(),score])

    consumption_output =[]
    for table in final_output['table']:
        table_details = {}
        table_details['coordinates'] = table[0]
        table_details['confidence'] = table[1]
        table_details['column'] = []
        table_details['header'] = []
        b1 = {}
        b1['y1'], b1['x1'], b1['y2'], b1['x2'] = table[0]
        for header in final_output['header']:
            b2 = {}
            b2['y1'], b2['x1'], b2['y2'], b2['x2'] = header[0]
            if get_iou(b1,b2)>0.05:
                out_header = {}
                out_header['coordinates'] = header[0]
                out_header['confidence'] = header[1]
                table_details['header'].append(out_header)

        for col in final_output['col']:
            c1={}
            col_out = {}
            c1['y1'], c1['x1'], c1['y2'], c1['x2'] = col[0]
            if get_iou(b1,c1)>0.05:
                col_out['coordinates'] = col[0]
                col_out['confidence'] = col[1]
                table_details['column'].append(col_out)

        consumption_output.append(table_details)

    return str(consumption_output)

In [11]:
import requests
#aptifyfull_3_.jpg
req = requests.Request()
req.files= {"file1" : open('/home/pcadmin/Desktop/project/AILogistics/mosaic-ai-client/examples/tensordata/aptifyfull_3_.jpg','rb')}

In [12]:
score(detection_graph,req)

"[{'coordinates': [0.14163130521774292, 0.0346657820045948, 0.41912317276000977, 0.8786301612854004], 'confidence': 0.999982, 'column': [{'coordinates': [0.18253648281097412, 0.5956850647926331, 0.3958061635494232, 0.7382981777191162], 'confidence': 0.99999964}, {'coordinates': [0.19028232991695404, 0.30118775367736816, 0.3950778841972351, 0.4318418502807617], 'confidence': 0.9999995}, {'coordinates': [0.19890160858631134, 0.43660441040992737, 0.3951108753681183, 0.5779754519462585], 'confidence': 0.9999976}, {'coordinates': [0.19285368919372559, 0.7353692650794983, 0.3925941586494446, 0.8562930226325989], 'confidence': 0.9999839}, {'coordinates': [0.18872244656085968, 0.06222282722592354, 0.3907977044582367, 0.17917387187480927], 'confidence': 0.9999478}, {'coordinates': [0.20323966443538666, 0.1858462393283844, 0.38711562752723694, 0.2916997969150543], 'confidence': 0.9994253}], 'header': [{'coordinates': [0.12847056984901428, 0.04787783324718475, 0.20246031880378723, 0.8682383298873

In [32]:
ndarray = np.array([1, 2, 3])
listing = ndarray.tolist()
print(listing)

[1, 2, 3]


In [28]:
register_model(detection_graph, score, "tensorflow_16", "testing temnsorflow model", MLModelFlavours.tensorflow)

{'created_by': 'akhil.lawrence',
 'created_on': '2019-10-20T13:39:36+00:00',
 'description': 'testing temnsorflow model',
 'flavour': 'tensorflow',
 'id': 'd2328cd9-2190-47f5-9340-c07f364fe706',
 'init_script': '',
 'last_modified_by': 'akhil.lawrence',
 'last_modified_on': '2019-10-20T13:39:36+00:00',
 'name': 'tensorflow_16',
 'project_id': '1',
 'versions': [{'created_by': 'akhil.lawrence',
   'created_on': '2019-10-20T13:39:37+00:00',
   'deployments': [],
   'description': None,
   'id': 'f366fd69-e644-4e3a-93b2-04116ac2e810',
   'last_modified_by': 'akhil.lawrence',
   'last_modified_on': '2019-10-20T13:39:44+00:00',
   'metadata_info': None,
   'ml_model_id': 'd2328cd9-2190-47f5-9340-c07f364fe706',
   'object_url': 'd2328cd9-2190-47f5-9340-c07f364fe706/f366fd69-e644-4e3a-93b2-04116ac2e810/ml_model.tar.gz',
   'profiling': [],
   'schema': None}]}

In [29]:
load_model("d2328cd9-2190-47f5-9340-c07f364fe706", "f366fd69-e644-4e3a-93b2-04116ac2e810")

(<tensorflow.python.framework.ops.Graph at 0x7fe1a77e36a0>,
 <mosaic_utils.ai.decorators.scoring_func at 0x7fe1a77e3f60>)

In [4]:
path = r'./frozen_inference_graph.pb'
def load_frozen_inference_graph(path):
    detection_graph = tf.Graph()
    with detection_graph.as_default():
      od_graph_def = tf.GraphDef()
      with tf.gfile.GFile(path, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
    return detection_graph


In [5]:
a=load_frozen_inference_graph(path)

In [6]:
a


In [7]:
print(a)